In [1]:
manufacturer = 'STANDARD MOTOR PRODUCTS'
type_code = '12036'

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()
list_part_number = df['Part_Number'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

import numpy as np

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url, part_number in zip(list_url, list_part_number):
    work.put_nowait((url, part_number))

def crawler():
    global df
    
    while not work.empty():
        url, part_number = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))
                
                if len(html.xpath('span[contains(@class, "navlabellink") and contains(@class, "nvoffset") and contains(@class, "nnormal")]/text()')) != 0 and html.xpath('span[contains(@class, "navlabellink") and contains(@class, "nvoffset") and contains(@class, "nnormal")]/text()')[0] == 'No Parts Found':
                    df_temp = pd.DataFrame([{'status': 'ok',
                                             'Manufacturer': manufacturer,
                                             'Part_Number': part_number,
                                             'Url': url}])
                    
                    status = 'ok'
                    
                    break

                list_part = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

                # = = = = = = = = = = = = = = =
                
                list_part_number = [_.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@title, "Buyer\'s Guide")]')[0].text for _ in list_part]
                
                # = = = = = = = = = = = = = = =
                
                number_index = np.where(np.array(list_part_number) == part_number)[0]
                
                # = = = = = = = = = = = = = = =
                
                list_manufacturer = [_.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "listing-final-manufacturer")]')[0].text for _ in list_part]
                
                # = = = = = = = = = = = = = = =
                
                manufacturer_index = np.where(np.array(list_manufacturer) == manufacturer)[0]
                
                # = = = = = = = = = = = = = = =
                
                index = list(np.intersect1d(number_index, manufacturer_index))
                
                # = = = = = = = = = = = = = = =
                
                if len(index) == 1:
                    index = index[0]
                    
                    # = = = = = = = = = = = = = = =
                    
                    note_1 = list_part[index].xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "span-link-underline-remover")]/text()')
                    if len(note_1) == 0:
                        note_1 = ''
                    else:
                        note_1 = note_1[0].strip()
   
                    # = = = = = = = = = = = = = = =
                    
                    note_2 = list_part[index].xpath('./descendant::div[contains(@class, "listing-text-row") and not(contains(@class, "moreinfo-truck"))]/span/span/text()')
                    if len(note_2) == 0:
                        note_2 = ''
                    else:
                        note_2 = [_.strip() for _ in note_2]
                        while '' in note_2:
                            note_2.remove('')
                        note_2 = '\n'.join(note_2)

                    # = = = = = = = = = = = = = = =
                    
                    oe = list_part[index].xpath('./descendant::span[contains(@title, "Replaces these Alternate/ OE Part Numbers")]')
                    if len(oe) == 0:
                        oe = ''
                    else:
                        oe = oe[0].xpath('./text()')[0].strip().replace(', ', ';')
                        
                    # = = = = = = = = = = = = = = =
                    
                    src = list_part[index].xpath('./descendant::img[contains(@id, "inlineimg_thumb")]')
                    if len(src) == 0:
                        src = ''
                    else:
                        src = 'https://www.rockauto.com' + src[0].xpath('./@src')[0].strip().replace('__ra_m.', '.')
    
                    # = = = = = = = = = = = = = = =
                    
                    part_json = list_part[index].xpath('./descendant::input[contains(@id, "listing_data_essential")]')
                    if len(part_json) == 0:
                        part_key = ''
                    else:
                        part_key = json.loads(part_json[0].xpath('./@value')[0])['partkey'].strip()
                        
                    # = = = = = = = = = = = = = = =
                    
                    df_temp = pd.DataFrame([{'status': 'ok',
                                             '序号': '',
                                             'Manufacturer': manufacturer,
                                             'Part_Number': part_number,
                                             'Vehcile': '',
                                             'Alternate_OE_Part_Numbers': oe,
                                             'Note_1': note_1,
                                             'Note_2': note_2,
                                             'Pic': '',
                                             'Url': url,
                                             'Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + type_code + '&pk=' + part_key,
                                             'Src': src,
                                             'Type_Code': type_code,
                                             'Part_Key': part_key}])
                
                    status = 'ok'
                
                    break
                
            except:
                time.sleep(0.5)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Manufacturer': manufacturer,
                                     'Part_Number': part_number,
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part_number +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：256

QS103M  <->  [ok] - 剩余数量：251
QS102M  <->  [ok] - 剩余数量：250
QS101R  <->  [ok] - 剩余数量：249
QS103R  <->  [ok] - 剩余数量：248
QS104M  <->  [ok] - 剩余数量：247
QS101M  <->  [ok] - 剩余数量：246
QS105M  <->  [ok] - 剩余数量：245
QS106M  <->  [ok] - 剩余数量：244
QS102R  <->  [ok] - 剩余数量：243
TPM101A  <->  [ok] - 剩余数量：242
TPM101RA  <->  [ok] - 剩余数量：241
QS106R  <->  [ok] - 剩余数量：240
TPM104A  <->  [ok] - 剩余数量：239
TPM104  <->  [ok] - 剩余数量：238
TPM103A  <->  [ok] - 剩余数量：237
TPM105A  <->  [ok] - 剩余数量：236
QS104R  <->  [ok] - 剩余数量：235
TPM105RA  <->  [ok] - 剩余数量：234
TPM108A  <->  [ok] - 剩余数量：233
TPM107A  <->  [ok] - 剩余数量：232
TPM114A  <->  [ok] - 剩余数量：231
TPM113A  <->  [ok] - 剩余数量：230
TPM106A  <->  [ok] - 剩余数量：229
TPM115A  <->  [ok] - 剩余数量：228
TPM116RA  <->  [ok] - 剩余数量：227
TPM117A  <->  [ok] - 剩余数量：226
TPM116A  <->  [ok] - 剩余数量：225
TPM118A  <->  [ok] - 剩余数量：224
TPM121  <->  [ok] - 剩余数量：223
TPM119  <->  [ok] - 剩余数量：222
TPM124  <->  [ok] - 剩余数量：221
TPM124A  <->  [ok] - 剩余数量：220
TPM109A  <->  [ok] - 剩余数量：219
TPM128  <->  

/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_53616/1785743158.py:96: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  number_index = np.where(np.array(list_part_number) == part_number)[0]
/var/folders/20/g09tvx4s7mdd977ps185l3_r0000gn/T/ipykernel_53616/1785743158.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  manufacturer_index = np.where(np.array(list_manufacturer) == manufacturer)[0]


TPM140  <->  [ok] - 剩余数量：211
TPM141  <->  [ok] - 剩余数量：210
TPM13A  <->  [ok] - 剩余数量：209
TPM142  <->  [ok] - 剩余数量：208
TPM142A  <->  [ok] - 剩余数量：207
TPM143  <->  [ok] - 剩余数量：206
TPM144A  <->  [ok] - 剩余数量：205
TPM147  <->  [ok] - 剩余数量：204
TPM146  <->  [ok] - 剩余数量：203
TPM145  <->  [ok] - 剩余数量：202
TPM133A  <->  [ok] - 剩余数量：201
TPM149A  <->  [ok] - 剩余数量：200
TPM149RA  <->  [ok] - 剩余数量：199
TPM150  <->  [ok] - 剩余数量：198
QS105R  <->  [ok] - 剩余数量：197
TPM151A  <->  [ok] - 剩余数量：196
TPM152A  <->  [ok] - 剩余数量：195
TPM153A  <->  [ok] - 剩余数量：194
TPM151RA  <->  [ok] - 剩余数量：193
TPM155A  <->  [ok] - 剩余数量：192
TPM154  <->  [ok] - 剩余数量：191
TPM167  <->  [ok] - 剩余数量：190
TPM166A  <->  [ok] - 剩余数量：189
TPM168  <->  [ok] - 剩余数量：188
TPM153  <->  [ok] - 剩余数量：187
TPM16A  <->  [ok] - 剩余数量：186
TPM170  <->  [ok] - 剩余数量：185
TPM171  <->  [ok] - 剩余数量：184
TPM167A  <->  [ok] - 剩余数量：183
TPM172  <->  [ok] - 剩余数量：182
TPM169  <->  [ok] - 剩余数量：181
TPM173  <->  [ok] - 剩余数量：180
TPM176  <->  [ok] - 剩余数量：179
TPM175  <->  [ok] - 剩余数量：178
